### Disclaimer

Distribution authorized to U.S. Government agencies and their contractors. Other requests for this document shall be referred to the MIT Lincoln Laboratory Technology Office.

This material is based upon work supported by the Under Secretary of Defense for Research and Engineering under Air Force Contract No. FA8702-15-D-0001. Any opinions, findings, conclusions or recommendations expressed in this material are those of the author(s) and do not necessarily reflect the views of the Under Secretary of Defense for Research and Engineering.

© 2019 Massachusetts Institute of Technology.

The software/firmware is provided to you on an As-Is basis

Delivered to the U.S. Government with Unlimited Rights, as defined in DFARS Part 252.227-7013 or 7014 (Feb 2014). Notwithstanding any copyright notice, U.S. Government rights in this work are defined by DFARS 252.227-7013 or DFARS 252.227-7014 as detailed above. Use of this work other than as specifically authorized by the U.S. Government may violate any copyrights that exist in this work.


### Treasure Hunt Challenge

This notebook uses [Stable Baselines](https://stable-baselines.readthedocs.io/en/master/) to train an agent for the [GOSEEK-Challenge](https://github.mit.edu/TESS/goseek-challenge). 

Proximal Policy Optimization is used to train an agent defined by a CNN-LSTM network. The agent's observations consist of RGB, segmentation, and depth images and relative pose. This, along with the reward function, is defined in the [GoSeekFullPerception](https://github.mit.edu/TESS/tesse-gym/blob/master/src/tesse_gym/tasks/goseek/goseek_full_perception.py#L30) [gym environment](https://gym.openai.com/). 


__Contents__
- [Configure Environment](#Configuration)
- [Define Model](#Define-the-Model)
- [Train Model](#Train-the-Model)
- [Visualize Results](#Visualize-Results)

In [1]:
from pathlib import Path

from gym import spaces
from stable_baselines.common.policies import CnnLstmPolicy
from stable_baselines.common.vec_env import SubprocVecEnv, DummyVecEnv
from stable_baselines import PPO2
from tesse.msgs import *

from tesse_gym import get_network_config
from tesse_gym.tasks.goseek import GoSeekFullPerception, decode_observations


/home/francesco/anaconda3/envs/goseek/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/francesco/anaconda3/envs/goseek/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/francesco/anaconda3/envs/goseek/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/francesco/anaconda3/envs/goseek

# Configuration

#### Set sim path

In [2]:
filename = Path("../../goseek-challenge/simulator/goseek-v0.1.4.x86_64")
assert filename.exists(), f"Must set a valid path!"

#### Set environment parameters


__Note__ To minimize training time during initial use, we've set `total_timestamps` and `n_environments` to 1e5 and 2 respectively. Setting `total_timestamps` to 3e6 and `n_environments` to 4 should produce an agent that approximates our baseline. 

In [3]:
n_environments = 5  # number of environments to train over
total_timesteps = 1000001  # number of training timesteps
scene_id = [1, 2, 3, 4, 5]  # list all available scenes
n_targets = 30  # number of targets spawned in each scene
target_found_reward = 2  # reward per found target
episode_length = 400


def make_unity_env(filename, num_env):
    """ Create a wrapped Unity environment. """

    def make_env(rank):
        def _thunk():
            env = GoSeekFullPerception(
                str(filename),
                network_config=get_network_config(worker_id=rank),
                n_targets=n_targets,
                episode_length=episode_length,
                scene_id=scene_id[rank%len(scene_id)],#np.random.choice(scene_id),
                target_found_reward=target_found_reward,
            )
            return env

        return _thunk

    return SubprocVecEnv([make_env(i) for i in range(num_env)])

#### Launch environments.

In [4]:
env = make_unity_env(filename, n_environments)

LOAD_MODEL = True

# Define the Model 

The following network assumes an observation of consisting of RGB, segmentation, and depth images along with the agent's relative pose from start. Images are processed using the Stable Baseline default CNN. The resulting feature vector is concatenated with the pose vector and given to an LSTM.

In [5]:
import tensorflow as tf
from stable_baselines.common.policies import nature_cnn

#### Define network to consume images and pose

In [6]:

def decode_tensor_observations(observation, img_shape=(-1, 240, 320, 5)):
    """ Decode observation vector into images and poses.

    Args:
        observation (np.ndarray): Shape (N,) observation array of flattened
            images concatenated with a pose vector. Thus, N is equal to N*H*W*C + N*3.
        img_shape (Tuple[int, int, int, int]): Shapes of all images stacked in (N, H, W, C).
            Default value is (-1, 240, 320, 5).
    
    Returns:
        Tuple[tf.Tensor, tf.Tensor]: Tensors with the following information
            - Tensor of shape (N, `img_shape[1:]`) containing RGB,
                segmentation, and depth images stacked across the channel dimension.
            - Tensor of shape (N, 3) containing (x, y, heading) relative to starting point.
                (x, y) are in meters, heading is given in degrees in the range [-180, 180].
    """
    
    imgs = tf.reshape(observation[:, :-3], img_shape)[..., -2:]
    pose = observation[:, -3:]
#     im1 = tf.image.resize(
#         imgs[..., :3], tf.constant([img_shape[1]//10, img_shape[2]//10], dtype=np.int32), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR
#     )
#     im2 = tf.image.resize(
#         tf.expand_dims(imgs[..., 3], axis=3), tf.constant([img_shape[1]//10, img_shape[2]//10], dtype=np.int32), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR
#     )
#     im3 = tf.image.resize(
#         tf.expand_dims(imgs[..., 4], axis=3), tf.constant([img_shape[1]//10, img_shape[2]//10], dtype=np.int32), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR
#     )

#     new_imgs = im2 #tf.concat([im1, im2, im3], axis=3)


#     return tf.reshape(new_imgs, [-1, new_imgs.shape[1]*new_imgs.shape[2]*new_imgs.shape[3]]), imgs, pose
    return imgs, pose

In [7]:

from stable_baselines.a2c.utils import conv, linear, conv_to_fc

def cnn(scaled_images, **kwargs):

    c1 = tf.nn.relu(conv(scaled_images, 'c1', n_filters=16, filter_size=4, stride=2, init_scale=np.sqrt(2), **kwargs))
    c2 = tf.nn.relu(conv(c1, 'c2', n_filters=24, filter_size=4, stride=2, init_scale=np.sqrt(2), **kwargs))
    c3 = tf.nn.relu(conv(c2, 'c3', n_filters=32, filter_size=3, stride=1, init_scale=np.sqrt(2), **kwargs))
#     c3 = tf.nn.l2_normalize(c3, axis=-1)

    g = tf.nn.relu(linear(conv_to_fc(c3), n_hidden=128, scope="g", init_scale=np.sqrt(2)))
    
    return g 

In [8]:
def features_combiner(image_features, pose):
    _, plen = pose.get_shape()
    pose_rs = tf.reshape(pose, (-1, 1, plen))
    _, featlen = image_features.get_shape()
    image_features_rs = tf.reshape(image_features, (-1, 1, featlen))
    
    ## memory for pose
    (pose_output, _, _) = tf.compat.v1.keras.layers.CuDNNLSTM(units=64, stateful=True, return_state=True)(pose_rs)
    print("pose_output", pose_output.get_shape())
    
    ## memory for image_features
    (img_output, _, _) = tf.compat.v1.keras.layers.CuDNNLSTM(units=256, stateful=True, return_state=True)(image_features_rs)
    print("img_output", img_output.get_shape())
    
    ## combined memory
    
    return  tf.concat((image_features, img_output, pose, pose_output), axis=-1)
    

In [9]:
def image_and_pose_network(observation, **kwargs):
    """ Network to process image and pose data.
    
    Use the stable baselines nature_cnn to process images. The resulting
    feature vector is then combined with the pose estimate and given to an
    LSTM (LSTM defined in PPO2 below).
    
    Args:
        raw_observations (tf.Tensor): 1D tensor containing image and 
            pose data.
        
    Returns:
        tf.Tensor: Feature vector. 
    """
    orig_imgs, pose = decode_tensor_observations(observation)
    scaled_imgs = tf.image.resize_images(orig_imgs, [40, 40], method=1) # 1: nearest
    image_features = cnn(scaled_imgs)
    combined_features = features_combiner(image_features, pose)
#     print(image_features.shape, imgs.shape, pose.shape)
    return combined_features

#### Register custom network

Outputs of the network defined above will be fed into an LSTM defined below in PPO2.

In [10]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [11]:
# policy_kwargs = {'cnn_extractor': image_and_pose_network}
policy_kwargs = {'cnn_extractor': image_and_pose_network}

In [12]:

if LOAD_MODEL:
    MODEL_WEIGHTS_PATH = "results/goseek-ppo-lstm-forwardreward/final_model.pkl"
    assert MODEL_WEIGHTS_PATH, f"Must give a model weights path!"
    model = PPO2.load(str(MODEL_WEIGHTS_PATH),env=env, tensorboard_log="./tensorboard/", gamma=0.995, learning_rate=0.0002)
else:
    model = PPO2(
        CnnLstmPolicy,
        env,
    #     n_steps=100,
        verbose=1,
        tensorboard_log="./tensorboard/",
        nminibatches=5,
        gamma=0.995,
        learning_rate=0.00025,
        policy_kwargs=policy_kwargs,
    )

Instructions for updating:
Colocations handled automatically by placer.
pose_output (5, 64)
img_output (5, 256)
pose_output (128, 64)
img_output (128, 256)
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


# Train the Model

#### Define logging directory and callback function to save checkpoints

In [13]:
log_dir = Path("results/goseek-ppo-lstm-forwardreward-retrain")
log_dir.mkdir(parents=True, exist_ok=True)

total_updates = 0
def save_checkpoint_callback(local_vars, global_vars):
    global total_updates
#     print(f"=== local vars ===\n{local_vars.keys()}")  # add this line 
#     total_updates = local_vars["n_updates"]
    total_updates += 1
    if total_updates % 1000 == 0:
        local_vars["self"].save(str(log_dir / f"{total_updates:06d}.pkl"))
        print("Saving model")

In [ ]:
if LOAD_MODEL:
    model.learn(total_timesteps=total_timesteps, tb_log_name='PPO2_75_retrain', callback=save_checkpoint_callback, reset_num_timesteps=False)
else:
    model.learn(total_timesteps=total_timesteps, callback=save_checkpoint_callback)

## TODO:
## add little penalty (-0.01) for using the grab action... because it grabs too much
## DONE add little reward (0.01) for going forward... bcs it rotates too much
## visualize attention?

-------------------------------------
| approxkl           | 0.0046764873 |
| clipfrac           | 0.044140626  |
| explained_variance | 0.532        |
| fps                | 11           |
| n_updates          | 1            |
| policy_entropy     | 1.0203629    |
| policy_loss        | -0.0141234   |
| serial_timesteps   | 128          |
| time_elapsed       | 0.000147     |
| total_timesteps    | 640          |
| value_loss         | 0.5051893    |
-------------------------------------
-------------------------------------
| approxkl           | 0.0057010315 |
| clipfrac           | 0.0625       |
| explained_variance | 0.668        |
| fps                | 22           |
| n_updates          | 2            |
| policy_entropy     | 0.8271537    |
| policy_loss        | -0.015637541 |
| serial_timesteps   | 256          |
| time_elapsed       | 56.4         |
| total_timesteps    | 1280         |
| value_loss         | 0.45759135   |
-------------------------------------
------------

-------------------------------------
| approxkl           | 0.0046719396 |
| clipfrac           | 0.049609374  |
| explained_variance | 0.664        |
| fps                | 27           |
| n_updates          | 18           |
| policy_entropy     | 0.8032257    |
| policy_loss        | -0.009070677 |
| serial_timesteps   | 2304         |
| time_elapsed       | 463          |
| total_timesteps    | 11520        |
| value_loss         | 1.0958241    |
-------------------------------------
-------------------------------------
| approxkl           | 0.0096365865 |
| clipfrac           | 0.07578125   |
| explained_variance | 0.697        |
| fps                | 24           |
| n_updates          | 19           |
| policy_entropy     | 0.81111336   |
| policy_loss        | -0.00876399  |
| serial_timesteps   | 2432         |
| time_elapsed       | 486          |
| total_timesteps    | 12160        |
| value_loss         | 0.19805641   |
-------------------------------------
------------

In [ ]:
model.save(str(log_dir) +"/final_model.pkl")

# Visualize Results

__Note__: Stable-Baselines requires that policy input dimensions be consistent across training and testing. Thus, the number of environments used for visualization must be a multiple of the number of environments used for training. The observation vector is then appropriately duplicated during inference. 

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

#### Load model

In [ ]:
MODEL_WEIGHTS_PATH = "results/goseek-ppo-lstm-forwardreward/final_model.pkl"
assert MODEL_WEIGHTS_PATH, f"Must give a model weights path!"

model = PPO2.load(str(MODEL_WEIGHTS_PATH))
n_train_envs = model.act_model.initial_state.shape[0]

#### Visualize all observed images

In [ ]:
obs = env.reset()
img_shape = (-1, 240, 320, 5)
imgs = np.reshape(obs[:, :-3], img_shape)[..., -2:]
print(imgs.shape)
rgb, segmentation, depth, pose = decode_observations(obs)
lstm_state = None

print(pose)

assert (
    n_train_envs % obs.shape[0] == 0
), f"The number of visualization environments must be a multiple of the training environments"

In [ ]:
fig, ax = plt.subplots(1, 3)
ax[0].imshow(rgb[0])
ax[1].imshow(segmentation[0])
ax[2].imshow(depth[0])

print(rgb[0].shape)
print(segmentation[0].shape)
print(depth[0].shape)

#### Run an episode and plot the first person agent view

In [ ]:
import cv2

# TODO:
# - check that the state is correct (segmentation and values of classes)
# - check the robot pose values
done = False
fig, ax = plt.subplots(1, obs.shape[0])
ax = [ax] if obs.shape[0] == 1 else ax

for i in range(episode_length):
    actions, lstm_state = model.predict(
        np.concatenate((n_train_envs // obs.shape[0]) * [obs]),
        state=lstm_state,
        deterministic=False
    )

    actions = actions[: obs.shape[0]]

    obs, reward, done, _ = env.step(actions)
#     print(actions, done, reward)

    plt.cla()
    rgb, segmentation, depth, pose = decode_observations(obs)
    

    for i in range(obs.shape[0]):
        print(reward)
#         print(segmentation[i][:,10])
        print(np.max(depth[i]), np.min(depth[i]))
        ax[i].imshow(cv2.resize(depth[i], dsize=(40,40), interpolation=cv2.INTER_NEAREST), vmin=0., vmax=1.)
#         ax[i].imshow(depth[i], vmin=0., vmax=1.)

    
    fig.canvas.draw()

obs = env.reset()
rgb, segmentation, depth, pose = decode_observations(obs)
lstm_state = None